In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
from pathlib import Path
import unidecode
import regex
import json

In [3]:
DIR = '/home/julio/repos/pubmed_processed_data/query_engine'
# DIR = '/home/julio/repos/pubmed_processed_data/dicts'


In [5]:
file_list = ['pubmed_' + str(i) + '.json' for i in range(2005,2022)]


In [7]:
# with open(os.path.join(DIR,'pmid2info_small.pickle'),'rb') as handle:
with open(os.path.join(DIR, 'pmid2details.pickle'), 'rb') as handle:
    dic = pickle.load(handle)


In [8]:
dic[1232]

{'title': 'Uptake and disposition of aldrin and dieldrin by isolated perfused rabbit lung.',
 'abstract': 'The uptake, metabolism, and release of aldrin and dieldrin by the lungs were studied by use of isolated perfused rabbit lungs that were artificially ventilated and perfused through the pulmonary artery. Both recirculating and single-pass experiments were conducted using an artificial medium as perfusate. Aldrin accumulated in the lung from the perfusate through two distinct phases of uptake: a rapid phase involving simple diffusion and nonspecific binding and a slower phase representing its metabolic turnover as dieldrin. Dieldrin was not metabolized but accumulated in the lungs by a saturable and a nonsaturable process. Single-pass experiments with aldrin indicated that the initial velocity of uptake could be fitted to one component and a constant representing the rate of metabolism. Uptake of dieldrin was biphasic: one phase independent of the perfusate concentration and the oth

In [10]:
#collecting pmids per year
year2pmids = {}
for pmid,v in dic.items():
    year = v['pubdate']
    if year not in year2pmids:
        year2pmids[year] = []
        year2pmids[year].append(pmid)
    else:
        year2pmids[year].append(pmid)


In [15]:
with open('/home/julio/repos/event_finder/data/year2pmid.pickle','wb') as handle:
    pickle.dump(year2pmids,handle)

In [7]:
texts = [v[4] for k,v in dic.items()]

In [29]:
values = dic.values()
values = [value[3] + value[4] for value in values]
values[0]

"[Biochemical studies on camomile components/III. In vitro studies about the antipeptic activity of (--)-alpha-bisabolol (author's transl)].(--)-alpha-Bisabolol has a primary antipeptic action depending on dosage, which is not caused by an alteration of the pH-value. The proteolytic activity of pepsin is reduced by 50 percent through addition of bisabolol in the ratio of 1/0.5. The antipeptic action of bisabolol only occurs in case of direct contact. In case of a previous contact with the substrate, the inhibiting effect is lost."

In [28]:
pmids = list(dic.keys())
pmids[0]

21

In [31]:
docs = texts[:100]
pmids = pmids[:100]

In [10]:
from bef.text_utils import preprocess_texts_DEM

In [11]:
import spacy

In [17]:
# nlp = spacy.load("en_core_web_sm", disable=['tagger', 'parser', 'ner'])
nlp = spacy.blank("en")
nlp.add_pipe('sentencizer')


In [21]:
newones=preprocess_texts_DEM(docs,nlp)

In [23]:
newones[0]

[' (  -  -  )  - alpha - Bisabolol has a primary antipeptic action depending on dosage ,  which is not caused by an alteration of the pH - value . ',
 'The proteolytic activity of pepsin is reduced by 50 percent through addition of bisabolol in the ratio of 1 / 0 . 5 . ',
 'The antipeptic action of bisabolol only occurs in case of direct contact . ',
 'In case of a previous contact with the substrate ,  the inhibiting effect is lost . ']

In [32]:
dict(zip(pmids,newones))

{21: [' (  -  -  )  - alpha - Bisabolol has a primary antipeptic action depending on dosage ,  which is not caused by an alteration of the pH - value . ',
  'The proteolytic activity of pepsin is reduced by 50 percent through addition of bisabolol in the ratio of 1 / 0 . 5 . ',
  'The antipeptic action of bisabolol only occurs in case of direct contact . ',
  'In case of a previous contact with the substrate ,  the inhibiting effect is lost . '],
 22: ['A report is given on the recent discovery of outstanding immunological properties in BA 1  [ N -  ( 2 - cyanoethylene )  - urea ]  having a  ( low )  molecular mass M  =  111 . 104 . ',
  'Experiments in 214 DS carcinosarcoma bearing Wistar rats have shown that BA 1 ,  at a dosage of only about 12 percent LD50  ( 150 mg kg )  and negligible lethality  ( 1 . 7 percent )  ,  results in a recovery rate of 40 percent without hyperglycemia and ,  in one test ,  of 80 percent with hyperglycemia . ',
  'Under otherwise unchanged conditions the

In [15]:
len(dic)  # 17641398 ,22984
 


22984

Removing accents prevents  deepventmine to get stuck

In [20]:

# accented_string = u'Poznań.'
# # accented_string is of type 'unicode'
# unaccented_string = unidecode.unidecode(accented_string)
# # unaccented_string contains 'Malaga'and is of type 'str'
# unaccented_string


'Poznan.'

In [9]:
for pmid,values in dic.items():
    with open(os.path.join('pubmed_small',str(pmid)+'.txt'),'w') as file:
        title = unidecode.unidecode(values[3])
        abstract = unidecode.unidecode(values[4])
        data = title + '\n' + abstract
        
        if len(data) > 20:
            file.write(data) # 3 title, 4 abstract




In [9]:
chunk_len = len(dic) / 4

In [27]:
#create dataset by divviding the data into 4 sets
for i in range(1,5):
    Path('pubmed_small_'+str(i) +'/text' ).mkdir(parents=True, exist_ok=True)


In [28]:
setn = 1
for i, (pmid, values) in enumerate(dic.items()):

    with open(os.path.join('pubmed_small_'+str(setn) + '/text', str(pmid)+'.txt'), 'w') as file:
        title = unidecode.unidecode(values[3])
        abstract = unidecode.unidecode(values[4])
        data = title + '\n' + abstract

        if len(data) > 20:
            file.write(data)  # 3 title, 4 abstract

    if i > chunk_len * setn:
        setn += 1


        


# Divide pubmed.json into yearly jsons

In [4]:
with open('/home/julio/repos/event_finder/data/pubmed/year2pmid.pickle','rb') as handle:
    year2pmids = pickle.load(handle)

In [5]:
with open('/home/julio/repos/event_finder/data/pubmed/pubmed.json', 'r') as handle:
    pmid2sentences = json.load(handle)


In [12]:
SAVEDIR = '/home/julio/repos/event_finder/data/pubmed/json_pmid2sents_year'
for year,pmids in year2pmids.items():
    new_dict = {pmid:pmid2sentences[str(pmid)]  for pmid in pmids if str(pmid) in pmid2sentences}
    with open(os.path.join(SAVEDIR,'pubmed_' + year +'.json'),'w') as handle:
        json.dump(new_dict,handle)
    # break
        

